# Die assembly

With gdsfactory you can easily go from a simple Component, to a Component with many components inside.

In the same way that you need to Layout for DRC (Design Rule Check) clean devices, you have to layout obeying the Design for Test (DFT) and Design for Packaging rules.

## Design for test

To measure your chips after fabrication you need to decide your test configurations. This includes Design For Testing Rules like:

- `Individual input and output fibers` versus `fiber array`. You can use `add_fiber_array` for easier testing and higher throughput, or `add_fiber_single` for the flexibility of single fibers.
- Fiber array pitch (127um or 250um) if using a fiber array.
- Pad pitch for DC and RF high speed probes (100, 125, 150, 200um). Probe configuration (GSG, GS ...)
- Test layout for DC, RF and optical fibers.



In [ ]:
from functools import partial

import gdsfactory as gf

gf.config.rich_output()

## Pack

Lets start with a resistance sweep, where you change the resistance width to measure sheet resistance.

In [ ]:
def add_resistance_sweep_info(c):
    c.info["doe"] = "resistance_sweep"
    c.info["analysis"] = "[iv_resistance]"
    c.info["analysis_parameters"] = "[{}]"
    c.info["ports_electrical"] = 2
    c.info["ports_optical"] = 0
    c.info["measurement"] = "iv"
    c.info["measurement_parameters"] = "{}"
    return c


sweep = [gf.components.resistance_sheet(width=width) for width in [1, 10, 50]]
sweep_with_info = [add_resistance_sweep_info(c) for c in sweep]
m = gf.pack(sweep_with_info)
c = m[0]
c.draw_ports()
c.pprint_ports()
c.plot()

In [ ]:
sweep_with_info[0].info

Then we add spirals with different lengths to measure waveguide propagation loss. You can use both fiber array or single fiber.

In [ ]:
@gf.cell
def spiral_gc(**kwargs):
    """Returns spiral with Grating Couplers."""
    c = gf.components.spiral(**kwargs)
    c = gf.routing.add_fiber_array(c)
    c.info["doe"] = "spirals_sc"  # strip Cband spirals
    c.info["measurement"] = "optical_spectrum"
    c.info["measurement_parameters"] = "{}"
    c.info["analysis"] = "[power_envelope]"
    c.info["analysis_parameters"] = "[]"
    c.info["ports_optical"] = 4
    c.info["ports_electrical"] = 0
    c.info.update(kwargs)
    return c


c = spiral_gc(length=100)
c.plot()

In [ ]:
c.info

In [ ]:
sweep = [spiral_gc(length=length) for length in [100, 200, 300]]
m = gf.pack(sweep)
c = m[0]
c.plot()

You can also add some physical labels that will be fabricated.
For example you can add prefix `S` at the `north-center` of each spiral using `text_rectangular` which is DRC clean and anchored on `nc` (north-center)

In [ ]:
text_metal = partial(gf.components.text_rectangular_multi_layer, layers=("M1",))

m = gf.pack(sweep, text=text_metal, text_anchors=("cw",), text_prefix="s")
c = m[0]
c.show()
c.plot()

## Grid

You can also pack components with a constant spacing.

In [ ]:
g = gf.grid(sweep)
g.plot()

In [ ]:
gh = gf.grid(sweep, shape=(1, len(sweep)))
gh.plot()

In [ ]:
gh_ymin = gf.grid(sweep, shape=(len(sweep), 1), align_x="xmin")
gh_ymin.plot()

You can also add text labels to each element of the sweep

In [ ]:
gh_ymin = gf.grid_with_text(
    sweep, shape=(len(sweep), 1), align_x="xmax", text=text_metal
)
gh_ymin.plot()

In [ ]:
gh_ymin = gf.grid_with_text(
    sweep,
    shape=(len(sweep), 1),
    align_x="xmax",
    text=text_metal,
    labels=("S100", "S200", "S300"),
)
gh_ymin.plot()

You have 2 ways of defining a mask:

1. in YAML
2. in Python

## YAML Component

You can also define your Component in YAML format thanks to `gdsfactory.read.from_yaml`

You need to define:

- instances
- placements
- routes (optional)

and you can leverage:

1. `pack_doe`
2. `pack_doe_grid`

`pack_doe` places components as compact as possible.

`pack_doe_grid` places each component on a regular grid

In [ ]:
pdk = gf.get_active_pdk()
pdk.register_cells(add_fiber_array=gf.routing.add_fiber_array)

c = gf.read.from_yaml(
    """
name: mask_compact

instances:
  rings:
    component: pack_doe
    settings:
      doe: ring_single
      settings:
        radius: [30, 50, 20, 40]
        length_x: [1, 2, 3]
      do_permutations: True
      function:
        function: add_fiber_array
        settings:
            fanout_length: 200


  mzis:
    component: pack_doe_grid
    settings:
      doe: mzi
      settings:
        delta_length: [10, 100]
      do_permutations: True
      spacing: [10, 10]
      function: add_fiber_array

placements:
  rings:
    xmin: 50

  mzis:
    xmin: rings,east
"""
)

c.show()
c.plot()

## Automated testing and analysis 

This is useful when you have a lot of components and you want to automate the testing process.

There are two main ways to define which components are testable:

1. Include a `doe` (Design of Experiments) field in the component.info dictionary, as well as all relevant test and analysis information.
2. Include a GDS label in all component test points. There are many ways to define test points, but the most common is to use a GDS label with the format `<elec/opt>-<number_of_ports>-<cell_name>`. This way you can easily extract all test points from the GDS file.


In [ ]:
import pandas as pd

import gdsfactory as gf


@gf.cell
def mzm_gc(length_x=10, **kwargs) -> gf.Component:
    """Returns a MZI with Grating Couplers.

    Args:
        length_x: length of the MZI.
        kwargs: additional settings.
    """
    c = gf.components.mzi2x2_2x2_phase_shifter(
        length_x=length_x, auto_rename_ports=False, **kwargs
    )
    c = gf.routing.add_pads_top(c, port_names=["top_l_e1", "top_r_e3"])
    c = gf.routing.add_fiber_array(c)
    c.info["doe"] = "mzm"
    c.info["measurement"] = "optical_spectrum"
    c.info["analysis"] = "[fsr]"
    c.info["analysis_parameters"] = "[]"
    c.info["ports_electrical"] = 2
    c.info["ports_optical"] = 6
    c.info["length_x"] = length_x
    c.info.update(kwargs)
    return c


def sample_reticle(grid: bool = False) -> gf.Component:
    """Returns MZI with TE grating couplers."""

    mzis = [mzm_gc(length_x=lengths) for lengths in [100, 200, 300]]
    spirals = [spiral_gc(length=length) for length in [0, 100, 200]]
    rings = []
    for length_x in [10, 20, 30]:
        ring = gf.components.ring_single_heater(length_x=length_x)
        c = gf.components.add_fiber_array_optical_south_electrical_north(
            component=ring,
            electrical_port_names=["l_e2", "r_e2"],
            grating_coupler=gf.components.grating_coupler_te, 
            pad=gf.components.pad,
            cross_section_metal='metal3'
        )
        c.name = f"ring_{length_x}"
        c.info["doe"] = "ring_length_x"
        c.info["measurement"] = "optical_spectrum"
        c.info["ports_electrical"] = 2
        c.info["ports_optical"] = 4
        c.info["analysis"] = "[fsr]"
        c.info["analysis_parameters"] = "[]"
        c.info["length_x"] = length_x
        rings.append(c)

    copies = 3  # number of copies of each component
    components = mzis * copies + rings * copies + spirals * copies
    if grid:
        return gf.grid(components)
    c = gf.pack(components)
    if len(c) > 1:
        c = gf.pack(c)[0]
    return c[0]


c = sample_reticle()
c.show()
c

In [ ]:
gf.labels.write_test_manifest(c, csvpath="sample_reticle.csv")
df = pd.read_csv("sample_reticle.csv")
df

You can see a test manifest example [here](https://docs.google.com/spreadsheets/d/1845m-XZM8tZ1tNd8GIvAaq7ZE-iha00XNWa0XrEOabc/edit#gid=233591479)

## Automated testing with labels

The GDS info is stored in the GDS file metadata and can be lost if the GDS file is modified with other tools that are not aware of the metadata. To avoid this, GDSFactory also supports a more traditional way of defining test points, using GDS labels. 

For example, lets say you want to label the rightmost port of a component with a GDS label `port_type-number_of_ports-cell_name`. You can do this with the following code:


In [ ]:
import gdsfactory as gf
from gdsfactory.typings import LayerSpec

layer_label = "TEXT"


def label_farthest_right_port(
    component: gf.Component, ports: gf.Port | list[gf.Port], layer: LayerSpec, text: str
) -> gf.Component:
    """Adds a label to the right of the farthest right port in a given component.

    Args:
        component: The component to which the label is added.
        ports: A list of ports to evaluate for positioning the label.
        layer: The layer on which the label will be added.
        text: The text to display in the label.
    """
    rightmost_port = max(ports, key=lambda port: port.dx)

    component.add_label(
        text=text,
        position=rightmost_port.dcenter,
        layer=layer,
    )
    return component


c = gf.Component()
ref = c << gf.routing.add_pads_top(gf.components.wire_straight())
label_farthest_right_port(c, ref.ports, layer=layer_label, text="elec-2-wire_straight")
c

In [ ]:
def spiral_gc(length: float = 0, **kwargs) -> gf.Component:
    """Returns a spiral double with Grating Couplers.

    Args:
        length: length of the spiral straight section.
        kwargs: additional settings.

    Keyword Args:
        bend: bend component.
        straight: straight component.
        cross_section: cross_section component.
        spacing: spacing between the spiral loops.
        n_loops: number of loops.
    """
    c0 = gf.c.spiral(length=length, **kwargs)
    c = gf.routing.add_fiber_array(c0)
    c.info["doe"] = "spirals_sc"
    c.info["measurement"] = "optical_spectrum"
    c.info["analysis"] = "[power_envelope]"
    c.info["analysis_parameters"] = "[]"
    c.info["ports_optical"] = 4
    c.info["ports_electrical"] = 0
    c.info.update(kwargs)

    c.name = f"spiral_gc_{length}"
    label_farthest_right_port(c, c.ports, layer=layer_label, text=f"opt-4-{c.name}")
    return c


c = spiral_gc(length=0)
c

In [ ]:
def mzi_gc(length_x=10, **kwargs) -> gf.Component:
    """Returns a MZI with Grating Couplers.

    Args:
        length_x: length of the MZI.
        kwargs: additional settings.
    """
    c = gf.components.mzi2x2_2x2_phase_shifter(
        length_x=length_x, auto_rename_ports=False, **kwargs
    )
    c = gf.routing.add_pads_top(c, port_names=("top_l_e1", "top_r_e3"))
    c.name = f"mzi_{length_x}"
    c = gf.routing.add_fiber_array(c)

    c.info["doe"] = "mzi"
    c.info["measurement"] = "optical_spectrum"
    c.info["analysis"] = "[fsr]"
    c.info["analysis_parameters"] = "[]"
    c.info["ports_electrical"] = 2
    c.info["ports_optical"] = 6
    c.info["length_x"] = length_x
    c.info.update(kwargs)

    c.name = f"mzi_gc_{length_x}"
    label_farthest_right_port(
        c,
        c.ports.filter(port_type="vertical_te"),
        layer=layer_label,
        text=f"opt-{c.info['ports_optical']}-{c.name}",
    )
    label_farthest_right_port(
        c,
        c.ports.filter(port_type="electrical"),
        layer=layer_label,
        text=f"elec-{c.info['ports_electrical']}-{c.name}",
    )
    return c


c = mzi_gc(length_x=10)
c

In [ ]:
def sample_reticle_with_labels(grid: bool = False) -> gf.Component:
    """Returns MZI with TE grating couplers."""

    mzis = [mzi_gc(length_x=lengths) for lengths in [100, 200, 300]]
    spirals = [spiral_gc(length=length) for length in [0, 100, 200]]
    rings = []
    for length_x in [10, 20, 30]:
        ring = gf.components.ring_single_heater(length_x=length_x)
        c = gf.components.add_fiber_array_optical_south_electrical_north(
            component=ring,
            electrical_port_names=["l_e2", "r_e2"],
            grating_coupler=gf.components.grating_coupler_te, 
            pad=gf.components.pad,
            cross_section_metal='metal3'
        )
        c.name = f"ring_{length_x}"
        c.info["doe"] = "ring_length_x"
        c.info["measurement"] = "optical_spectrum"
        c.info["ports_electrical"] = 2
        c.info["ports_optical"] = 4
        c.info["analysis"] = "[fsr]"
        c.info["analysis_parameters"] = "[]"
        label_farthest_right_port(
            c,
            c.ports.filter(port_type="vertical_te"),
            layer=layer_label,
            text=f"opt-{c.info['ports_optical']}-{c.name}",
        )
        label_farthest_right_port(
            c,
            c.ports.filter(port_type="electrical"),
            layer=layer_label,
            text=f"elec-{c.info['ports_electrical']}-{c.name}",
        )
        rings.append(c)

    copies = 3  # number of copies of each component
    components = mzis * copies + rings * copies + spirals * copies
    if grid:
        return gf.grid(components)
    c = gf.pack(components)
    if len(c) > 1:
        c = gf.pack(c)[0]
    return c[0]


c = sample_reticle_with_labels()
c

You can also extract all test points from a GDS file using gf.labels.write_labels

In [ ]:
import pandas as pd

gdspath = c.write_gds()
csvpath = gf.labels.write_labels(gdspath, layer_label=layer_label)
df = pd.read_csv(csvpath)
df = df.sort_values(by=["text"])
df